In [1]:
# import
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.cluster import DBSCAN

# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티
# DBSCAN : 밀도 기반 클러스터링 : 점 p에서 부터 거리 e (epsilon)내에 점이 m(minPts) 개 있으면 하나의 군집으로 인식

In [2]:
# KST = timezone('Asia/Seoul')    # 서울 시간

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
# browser.implicitly_wait(1)

# 전처리 함수

In [63]:
# 필요없는 내용 삭제 함수
def clean(article):
    article = re.sub('\w{2,4}기자','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\[.{1,15}\]','',article)
    article = re.sub('\w+ 기상캐스터','',article)
    article = re.sub('사진','',article)
    article = re.sub('포토','',article)
    article = re.sub('\(.*뉴스.{0,3}\)','', article)  # (~뉴스~) 삭제
    article = re.sub('\S+@[a-z.]+','',article)          # 이메일 삭제
    article = re.sub("(\s=\s)","", article)

    article = re.sub('[\n\t\u200b\xa0]','',article)
    # article = re.sub('\n','',article)
    # article = re.sub('\t','',article)
    # article = re.sub('\u200b','',article)
    # article = re.sub('\xa0','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    # article = re.sub('([a-zA-Z])','',article)   # 영어 삭제
    # article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘’“”|\(\)\[\]\<\>`\'…》]','',article)   # 특수문자 삭제

    return article


# 본문에서 명사 뽑아내는 함수
def getNouns(article_df):
    okt = Okt()
    nouns_list = []                               # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.nouns(content))     # 명사 추출 (리스트 반환)

    article_df["nouns"] = nouns_list              # 데이터 프레임에 추가

    return article_df

# 명사를 벡터화 하는 함수
def getVector(article_df):    # 카테고리 별로 벡터 생성
    category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]
    vector_list = []

    for i in range(8):
        try:
            text = [" ".join(noun) for noun in article_df['nouns'][article_df['category'] == category_names[i]]]    # 명사 열을 하나의 리스트에 담는다.

            tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 5))
            tfidf_vectorizer.fit(text)
            vector = tfidf_vectorizer.transform(text).toarray()                         # vector list 반환
            vector = np.array(vector)
            vector_list.append(vector)
        except:
            print("크롤링 안 한 카테고리 :", category_names[i])

    return vector_list

def convertCategory(article_df):    # 이름으로된 카테고리를 번호로 변환
    category = [("정치", "100"), ("경제", "101"), ("사회", "102"), ("생활/문화", "103"), ("세계", "104"), ("IT/과학", "105"), ("연예", "106"), ("스포츠", "107")]

    for name, num in category:
        article_df["category"][article_df["category"] == name] = num

    return article_df

def removeEnglishArticle(article_df):   # 영어 기사 삭제
    index = article_df[article_df['nouns'].apply(len) <= 5].index
    return article_df.drop(index, inplace=True)

## 크롤링 클래스

In [8]:
# 기사 링크 크롤링
class UrlCrawling:
    def __init__(self):
        self.category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]
        self.category = []

    def getSixUrl(self):    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
        six_url = []
        for category in range(6):     # 6
            a_list = []
            for page in range(1, 6):  # 1, 6
                url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
                browser.get(url)

                time.sleep(0.5)

                soup = BeautifulSoup(browser.page_source, "html.parser")
                a_list.extend(soup.select(".type06_headline dt+dt a"))
                a_list.extend(soup.select(".type06 dt+dt a"))

                print(f"{self.category_names[category]} {page} 페이지")

            for a in a_list:
                six_url.append(a["href"])
                self.category.append(self.category_names[category])

        return six_url


    def getEntertainmentUrl(self):   # 연예
        # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
        entertainment_url = []
        a_list = []
        today = datetime.date.today()

        for page in range(1, 2):  # 1, 5
            url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            a_list.extend(soup.select(".news_lst li>a"))


            print(f"연예 {page} 페이지")

        for a in a_list:
            entertainment_url.append("https://entertain.naver.com" + a["href"])
            self.category.append("연예")

        return entertainment_url

    def getSportsUrl(self):    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
        # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
        sports_url = []
        a_list = []
        today = str(datetime.date.today()).replace('-', '')

        for page in range(1, 2):  # 1, 7
            url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".news_list li>a"))

            print(f"스포츠 {page} 페이지")

        for i in range(len(a_list)):
            if i == 100:  # 100개 링크 추가했으면 멈추기
                break
            sports_url.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
            self.category.append("스포츠")

        return sports_url



# 기사 본문 크롤링
class ContentCrawling:
    def __init__(self, title, content, date, img):
        self.title = title
        self.content = content
        self.date = date
        self.img = img

    def getSixContent(self, url_list):  # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
        title_list = []
        content_list = []
        date_list = []
        img_list = []
        cnt = 1

        for url in url_list:
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            print(cnt)
            cnt+=1

            try:
                title_list.extend(soup.select("#title_area span"))              # 제목 추가

                c = soup.find_all(attrs={"id" : "dic_area"})                    # 본문 가져오기

                img_tag = soup.select(".end_photo_org img")                     # 이미지 가져오기

                if img_tag:                                                     # 이미지 있으면 이미지 주소만 추출해서 리스트로 만든다.
                    img_src_list = []
                    for img in img_tag:
                        img_src_list.append(img['src'])
                    img_list.append(",".join(img_src_list))
                else:
                    img_list.append("")

                while c[0].find(attrs={"class" : "end_photo_org"}):             # 이미지 있는 만큼
                    c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

                while c[0].find(attrs={"class" : "vod_player_wrap"}):           # 영상 있는 만큼
                    c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

                if c[0].find(attrs={"class" : "artical-btm"}):                  # 하단에 제보하기 칸 있으면 삭제
                    c[0].find(attrs={"class" : "artical-btm"}).decompose()

                content_list.extend(c)                                          # 본문 추가

                date_list.extend(soup.select("._ARTICLE_DATE_TIME"))            # 날짜 추가

            except IndexError:
                print("삭제된 기사")

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            self.date.append(d.text)

        for i in img_list:
            self.img.append(i)

    def getEntertainmentContent(self, url_list):    # 연예
        title_list = []
        content_list = []
        date_list = []
        img_list = []
        cnt = 1

        for url in url_list:
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            print(cnt)
            cnt+=1

            try:
                title_list.extend(soup.select(".end_tit"))                      # 제목 추가

                c = soup.find_all(attrs={"class" : "article_body"})             # 본문 가져오기

                img_tag = soup.select(".end_photo_org img")                     # 이미지 가져오기

                if img_tag:                                                     # 이미지 있으면 이미지 주소만 추출해서 리스트로 만든다.
                    img_src_list = []
                    for img in img_tag:
                        img_src_list.append(img['src'])
                    img_list.append(",".join(img_src_list))
                else:
                    img_list.append("")


                while c[0].find(attrs={"class" : "end_photo_org"}):             # 이미지 있는 만큼
                    c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

                if c[0].find(attrs={"class" : "caption"}):                      # 이미지 설명 없애기
                    c[0].find(attrs={"class" : "caption"}).decompose()

                while c[0].find(attrs={"id" : "video_area"}):                # 영상 있는 만큼
                    c[0].find(attrs={"id" : "video_area"}).decompose()       # 본문 영상 없애기

                while c[0].find(attrs={"name" : "iframe"}):
                    c[0].find(attrs={"name" : "iframe"}).decompose()

                content_list.extend(c)                                          # 본문 추가

                date_list.extend(soup.select_one(".author em"))                 # 날짜 추가

            except IndexError:
                print("삭제된 기사")

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            self.date.append(d.text)
            
        for i in img_list:
            self.img.append(i)

    def getSportsContent(self, url_list):   # 스포츠
        title_list = []
        content_list = []
        date_list = []
        img_list = []
        cnt = 1

        for url in url_list:

            browser.get(url)                                                    
            
            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            print(cnt)
            cnt+=1

            title_list.extend(soup.select(".news_headline .title"))             # 제목 추가 

            c = soup.find_all(attrs={"class" : "news_end"})                     # 본문 가져오기

            img_tag = soup.select(".end_photo_org img")                     # 이미지 가져오기

            if img_tag:                                                     # 이미지 있으면 이미지 주소만 추출해서 리스트로 만든다.
                img_src_list = []
                for img in img_tag:
                    img_src_list.append(img['src'])
                img_list.append(",".join(img_src_list))
            else:
                img_list.append("")

            while c[0].find(attrs={"class" : "end_photo_org"}):                 # 이미지 있는 만큼
                c[0].find(attrs={"class" : "end_photo_org"}).decompose()        # 본문 이미지에 있는 글자 없애기

            while c[0].find(attrs={"class" : "image"}):
                c[0].find(attrs={"class" : "image"}).decompose()

            while c[0].find(attrs={"class" : "vod_area"}):                      # 영상 있는 만큼
                c[0].find(attrs={"class" : "vod_area"}).decompose()             # 본문 영상 없애기

            if c[0].find(attrs={"class" : "source"}): c[0].find(attrs={"class" : "source"}).decompose()
            if c[0].find(attrs={"class" : "byline"}): c[0].find(attrs={"class" : "byline"}).decompose()
            if c[0].find(attrs={"class" : "reporter_area"}): c[0].find(attrs={"class" : "reporter_area"}).decompose()
            if c[0].find(attrs={"class" : "copyright"}): c[0].find(attrs={"class" : "copyright"}).decompose()
            if c[0].find(attrs={"class" : "categorize"}): c[0].find(attrs={"class" : "categorize"}).decompose()
            if c[0].find(attrs={"class" : "promotion"}): c[0].find(attrs={"class" : "promotion"}).decompose()

            content_list.extend(c)                                        # 본문 추가

            date_list.extend(soup.select_one(".info span"))               # 날짜 추가

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            d = (d.text)[5:]
            self.date.append(d)

        for i in img_list:
            self.img.append(i)

    def makeDataFrame(self, all_url, category):    # 수집한 데이터를 데이터프레임으로 변환

        article_df = pd.DataFrame({"category" : category,
                                   "date" : self.date,
                                   "title" : self.title,
                                   "content" : self.content,
                                   "img" : self.img,
                                   "url" : all_url})

        return article_df

# 군집화

**DBSCAN**
- https://bcho.tistory.com/1205


In [5]:
# 카테고리 별로 군집화
# cluster_number 열에 군집 번호 생성
def addClusterNumber(df, vector_list):
    cluster_number_list = []

    for vector in vector_list:
        model = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
        result = model.fit_predict(vector)
        cluster_number_list.extend(result)

    df['cluster_number'] = cluster_number_list  # 군집 번호 칼럼 추가


def getClusteredArticle(df): # 카테고리 별로 군집의 개수를 센다.
    category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]
    cluster_counts_df = pd.DataFrame({'category' : [""],
                                    'cluster_number' : [0],
                                    'cluster_count' : [0]})

    for i in range(8):
        t = df[df['category'] == category_names[i]]['cluster_number'].value_counts().reset_index()
        t.columns = ['cluster_number', 'cluster_count']
        t['category'] = [category_names[i]] * len(t)

        cluster_counts_df = pd.concat([cluster_counts_df, t])

    cluster_counts_df = cluster_counts_df[cluster_counts_df['cluster_count'] != 0]

    # 상위 군집 10개씩만 추출
    cluster_counts_df = cluster_counts_df[cluster_counts_df.index < 10]

    return cluster_counts_df

# 요약

In [6]:
from gensim.summarization.summarizer import summarize
import pandas as pd
# from summa.summarizer import summarize

def getSummaryArticle(article_df, cluster_counts_df):
    summary_article = pd.DataFrame(columns=["category", "title", "content", "img", "url"])

    for i in range(len(cluster_counts_df)):
        category_name, cluster_number = cluster_counts_df.iloc[i, 0:2]    # 카테고리 이름, 군집 번호

        temp_df = article_df[(article_df['category'] == category_name) & (article_df['cluster_number'] == cluster_number)]

        category = temp_df["category"].iloc[0]          # 카테고리
        title = temp_df["title"].iloc[0]                # 일단은 첫 번째 뉴스 제목
        content = "".join(temp_df["content"])           # 본문 내용 여러개를 하나의 문자열로 합쳐서 요약
        img = ",".join(list(temp_df["img"]))            # 전체 이미지
        url = ",".join(list(temp_df["url"]))            # 전체 링크

        try:
            summary_content = summarize(content, ratio=0.2)
            if not summary_content:     # 요약문이 비어있으면 (너무 짧아서?)
                summary_content = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
        except:
            summary_content = content
        finally:
            summary_article = summary_article.append({
                "category": category,
                "title": title,
                "content": summary_content,
                "img": img,
                "url": url
            }, ignore_index=True)

    return summary_article

# 데이터베이스 연동

In [65]:
import cx_Oracle as cx
import pandas as pd

# category, title, content, url
id = "c##2201058"
pw = "p2201058"
url = "10.30.3.95:1521/orcl"

conn = cx.connect(id, pw, url)

def insert(summary_article):

    sql = """insert into news(news_id, cate_id, title, content, img, link, views)
             values(news_id_seq.nextval, :1, :2, :3, :4, :5, 0)"""

    cur = conn.cursor()
    cur.executemany(sql, summary_article)

    cur.close()
    conn.commit()
    conn.close()

def select():

    sql = """select * from news
             order by to_number(news_id)"""
    
    cur = conn.cursor()
    cur.execute(sql)

    df = pd.read_sql(sql, con = conn)
    df["CONTENT"] = df["CONTENT"].astype("string")      # CLOB 데이터 타입을 string로 변경해야 df로 가져올 수 있음

    cur.close()
    conn.close()

    return df

# MAIN (코드 실행 페이지)

In [ ]:
# 링크 크롤링하는 객체 생성
url_crawler = UrlCrawling()

six_url = url_crawler.getSixUrl()                          # 6개 카테고리 url
entertainment_url = url_crawler.getEntertainmentUrl()      # 연예 url
sports_url = url_crawler.getSportsUrl()                    # 스포츠 url
all_url = six_url + entertainment_url + sports_url        # 전체 url
category = url_crawler.category                            # 카테고리 리스트

# 본문 크롤링하는 객체 생성
content_crawler = ContentCrawling([], [], [], [])

content_crawler.getSixContent(six_url)
content_crawler.getEntertainmentContent(entertainment_url)
content_crawler.getSportsContent(sports_url)

In [10]:
article_df = content_crawler.makeDataFrame(all_url, category)     # 본문 데이터프레임 생성

article_df = getNouns(article_df)                                 # 명사 추출

vector_list = getVector(article_df)                               # 명사 벡터화

addClusterNumber(article_df, vector_list)                         # 군집 번호 열 생성
cluster_counts_df = getClusteredArticle(article_df)               # 군집 개수 카운트한 df

summary_article = getSummaryArticle(article_df, cluster_counts_df)     # 요약한 기사 데이터 프레임 반환
summary_article = convertCategory(summary_article)                     # 카테고리 이름을 번호로 변환

NameError: name 'all_url' is not defined

In [66]:
# insert(summary_article.values.tolist())
select()

,NEWS_ID,CATE_ID,TITLE,CONTENT,IMG,LINK,VIEWS
0,1,100,"北, 9·19 합의로 파괴한 GP에 병력·중화기 투입…경계초소 설치""","군 관계자 ""파괴·철수한 11개 GP서 유사한 상황…해안포 개방도 많이 늘어""북한군...",https://imgnews.pstatic.net/image/087/2023/11/...,https://n.news.naver.com/mnews/article/087/000...,0
1,2,100,"'서울 험지' 출마한다던 하태경, 같은 당 최재형 지역구 종로로","말린다고 되겠나""부산 지역구를 버리고 차기 총선에 서울 험지 출마를 선언한 국민의힘...",https://imgnews.pstatic.net/image/002/2023/11/...,https://n.news.naver.com/mnews/article/002/000...,0
2,3,100,군 “북 위성체 궤도진입 성공… 위성기술은 아직”,"북 '한반도·하와이' 촬영 주장엔 ""보여주기식 선전""북한이 최근 발사한 군사정찰위성...",https://imgnews.pstatic.net/image/277/2023/11/...,https://n.news.naver.com/mnews/article/277/000...,0
3,4,100,"극단적 정쟁에 갇힌 정치, 사람 교체가 답",그동안 장 의원을 비롯한 국민의힘 영남권 중진 의원들은 인요한 혁신위원장으로부터 '...,https://imgnews.pstatic.net/image/586/2023/11/...,https://n.news.naver.com/mnews/article/586/000...,0
4,5,100,"민주, '권리당원 비중 강화' 당무위 의결…비명계 반발",권칠승 수석대변인은 이날 오전 국회에서 열린 당무위 들과 만나 전당대회 후보자 본선...,https://imgnews.pstatic.net/image/277/2023/11/...,https://n.news.naver.com/mnews/article/277/000...,0
5,6,100,"국정원장, 조직통솔·기강확립·방첩강화 차원서 '외부인사' 무게","천영우·김용현·권영세 등 거론…대통령실 ""늦어도 한 달 내 지명""내부 발탁 가능성도...",https://imgnews.pstatic.net/image/001/2023/11/...,https://n.news.naver.com/mnews/article/001/001...,0
6,7,100,'김기현 신경전'에 '집안 싸움'까지…與 혁신위 운명은?,사퇴를 표명했던 외부 영입 위원들과 정치인 출신 위원들의 생각이 여전히 다른 가운데...,https://imgnews.pstatic.net/image/586/2023/11/...,https://n.news.naver.com/mnews/article/586/000...,0
7,8,100,정권교체때마다 불거진 ‘국정원 권력투쟁’ 뿌리뽑는다,"■ 윤 대통령, 국정원 전면쇄신 방침권력다툼 인사는 전면배제키로진흙탕 파벌대립 위험...",https://imgnews.pstatic.net/image/021/2023/11/...,https://n.news.naver.com/mnews/article/021/000...,0
8,9,100,北 '9·19합의 파기' 선언 뒤 DMZ 내 초소 복구·중화기 배치,"軍 ""기존 시설물 복원 나선 듯… 서해 해안포 개방도 늘어"" 박응진 = 북한군이 ...",https://imgnews.pstatic.net/image/421/2023/11/...,https://n.news.naver.com/mnews/article/421/000...,0
9,10,100,"한동훈·이정재, 함께 식사...SNS 올라온 '인증샷' 화제",배우 이정재 씨 만나서 함께 식사한 장면을 시민들이 을 찍어서 SNS에 올리면서 화...,None,https://n.news.naver.com/mnews/article/052/000...,0


In [ ]:
# csv로 저장
# article_df.to_csv("test.csv",index=False, encoding="utf-8-sig")